In [1]:
import molsysmt as msm

In [2]:
msm.form.file_h5.attributes

{'atom_index': False,
 'atom_name': False,
 'atom_id': False,
 'atom_type': False,
 'group_index': False,
 'group_name': False,
 'group_id': False,
 'group_type': False,
 'component_index': False,
 'component_name': False,
 'component_id': False,
 'component_type': False,
 'chain_index': False,
 'chain_name': False,
 'chain_id': False,
 'chain_type': False,
 'molecule_index': False,
 'molecule_name': False,
 'molecule_id': False,
 'molecule_type': False,
 'entity_index': False,
 'entity_name': False,
 'entity_id': False,
 'entity_type': False,
 'bond_index': False,
 'bond_id': False,
 'bond_type': False,
 'bond_order': False,
 'bonded_atoms': False,
 'inner_bonded_atoms': False,
 'inner_bond_index': False,
 'n_atoms': False,
 'n_groups': False,
 'n_components': False,
 'n_chains': False,
 'n_molecules': False,
 'n_entities': False,
 'n_bonds': False,
 'n_inner_bonds': False,
 'n_aminoacids': False,
 'n_nucleotides': False,
 'n_ions': False,
 'n_waters': False,
 'n_small_molecules': Fal